<a href="https://colab.research.google.com/github/dakshnpci/NPCIMLTraining/blob/master/Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# !pip install hyperopt
# !pip install catboost
!pip install lightgbm

    100% |████████████████████████████████| 1.2MB 7.9MB/s 


In [1]:
#!/usr/bin/env python -W ignore::DeprecationWarning
 
import numpy as np                                                 # Implemennts milti-dimensional array and matrices
import pandas as pd                                                # For data manipulation and analysis
import matplotlib.pyplot as plt                                    # Plotting library for Python programming language and it's numerical mathematics extension NumPy
import seaborn as sns                                              # Provides a high level interface for drawing attractive and informative statistical graphics
%matplotlib inline
sns.set()
from collections import Counter
from subprocess import check_output
titanic_data = pd.read_csv("https://raw.githubusercontent.com/insaid2018/Term-1/master/Data/Casestudy/titanic_train.csv")     # Importing training dataset using pd.read_csv

def detect_outliers(df,n,features):
    """
    Takes a dataframe df of features and returns a list of the indices
    corresponding to the observations containing more than n outliers according
    to the Tukey method.
    """
    outlier_indices = []
    
    # iterate over features(columns)
    for col in features:
        # 1st quartile (25%)
        Q1 = np.percentile(df[col], 25)
        # 3rd quartile (75%)
        Q3 = np.percentile(df[col],75)
        # Interquartile range (IQR)
        IQR = Q3 - Q1
        
        # outlier step
        outlier_step = 1.5 * IQR
        
        # Determine a list of indices of outliers for feature col
        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step )].index
        
        # append the found outlier indices for col to the list of outlier indices 
        outlier_indices.extend(outlier_list_col)
        
    # select observations containing more than 2 outliers
    outlier_indices = Counter(outlier_indices)        
    multiple_outliers = list( k for k, v in outlier_indices.items() if v > n )
    
    return multiple_outliers   
  
# Get the outlier out (2 per column)
Outliers_to_drop = detect_outliers(titanic_data,2,["Age","SibSp","Parch","Fare"])
# print(titanic_data.loc[Outliers_to_drop]) # Show the outliers rows)
titanic_data = titanic_data.drop(Outliers_to_drop, axis = 0).reset_index(drop=True)

# Get the title from name
# Get Title from Name
titanic_data_title = [i.split(",")[1].split(".")[0].strip() for i in titanic_data["Name"]]
titanic_data["Title"] = pd.Series(titanic_data_title)


# Convert to categorical values Title 
titanic_data["Title"] = titanic_data["Title"].replace(['Lady', 'the Countess','Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
titanic_data["Title"] = titanic_data["Title"].map({"Master":0, "Miss":1, "Ms" : 1 , "Mme":1, "Mlle":1, "Mrs":1, "Mr":2, "Rare":3})
titanic_data["Title"] = titanic_data["Title"].astype(int)

# Add Cabin Details 
titanic_data["Cabin"] = pd.Series([i[0] if not pd.isnull(i) else 'X' for i in titanic_data['Cabin'] ])
titanic_data = pd.get_dummies(titanic_data, columns = ["Cabin"],prefix="Cabin")

titanic_data.Embarked = titanic_data.Embarked.fillna(titanic_data['Embarked'].mode()[0])
median_age = titanic_data.Age.median()
median_fare = titanic_data.Fare.median()
titanic_data.Age.fillna(median_age, inplace = True)
# titanic_data.drop('Cabin', axis = 1,inplace = True)
titanic_data.Fare.fillna(median_fare, inplace = True)
titanic_data['FamilySize'] = titanic_data['SibSp'] + titanic_data['Parch']+1
titanic_data['GenderClass'] = titanic_data.apply(lambda x: 'child' if x['Age'] < 15 else x['Sex'],axis=1)
titanic_data = pd.get_dummies(titanic_data, columns=['GenderClass','Embarked'], drop_first=True)
titanic = titanic_data.drop(['Name','Ticket','Sex','SibSp','Parch','PassengerId'], axis = 1)
titanic.head()
X = titanic.loc[:,titanic.columns != 'Survived']
y = titanic.Survived 
# X.head()

from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:4291: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)
/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
X_train.head(5)

,Pclass,Age,Fare,Title,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_X,FamilySize,GenderClass_female,GenderClass_male,Embarked_Q,Embarked_S
181,1,28.0,50.0000,2,1,0,0,0,0,0,0,0,0,1,0,1,0,1
329,3,28.0,7.8958,2,0,0,0,0,0,0,0,0,1,1,0,1,0,1
265,1,28.0,31.0000,2,0,0,0,0,0,0,0,0,1,1,0,1,0,1
501,3,28.0,22.5250,2,0,0,0,0,0,0,0,0,1,1,0,1,0,1
855,2,42.0,13.0000,1,0,0,0,0,0,0,0,0,1,1,1,0,0,1


In [5]:
import hyperopt
from catboost import CatBoostClassifier, Pool, cv

train_pool = Pool(X_train, y_train)

def hyperopt_objective(params):
    model = CatBoostClassifier(
        l2_leaf_reg=int(params['l2_leaf_reg']),
        learning_rate=params['learning_rate'],
        depth=params['depth'],
        iterations=1000,
        eval_metric='Accuracy',
        od_type='Iter',
        od_wait=40,
        random_seed=42,
        logging_level='Silent',
        allow_writing_files=False
    )
    
    cv_data = cv(
        train_pool,
        model.get_params()
    )
    best_accuracy = np.max(cv_data['test-Accuracy-mean'])    
    
    print(params, best_accuracy)
    return 1 - best_accuracy # as hyperopt minimises

params_space = {
    'l2_leaf_reg': hyperopt.hp.qloguniform('l2_leaf_reg', 0, 2, 1),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 5e-1),
    'depth': hyperopt.hp.choice('depth', [3,4,5,6,8]),
}

trials = hyperopt.Trials()

best = hyperopt.fmin(
    hyperopt_objective,
    space=params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=100,
    trials=trials
)

print("+++++++++++++++++++++++++++++++++++++++++++++++++++++")
print(best)


{'depth': 8, 'l2_leaf_reg': 7.0, 'learning_rate': 0.4784362672900233} 0.8310056373886162
{'depth': 4, 'l2_leaf_reg': 1.0, 'learning_rate': 0.11331509293216656} 0.8352730799539311
{'depth': 3, 'l2_leaf_reg': 2.0, 'learning_rate': 0.2700788249930544} 0.8324240771049282
{'depth': 4, 'l2_leaf_reg': 7.0, 'learning_rate': 0.425445593868832} 0.8324240771049282
{'depth': 4, 'l2_leaf_reg': 1.0, 'learning_rate': 0.015460514451838318} 0.8338425168212402
{'depth': 6, 'l2_leaf_reg': 4.0, 'learning_rate': 0.02315768668580303} 0.8310056373886162
{'depth': 3, 'l2_leaf_reg': 1.0, 'learning_rate': 0.005014341984825305} 0.81677880826817
{'depth': 6, 'l2_leaf_reg': 2.0, 'learning_rate': 0.3049078736545175} 0.8310056373886162
{'depth': 6, 'l2_leaf_reg': 2.0, 'learning_rate': 0.31200021061471783} 0.8310056373886162
{'depth': 6, 'l2_leaf_reg': 7.0, 'learning_rate': 0.2502614502341035} 0.8423652785354913
{'depth': 3, 'l2_leaf_reg': 7.0, 'learning_rate': 0.4771127636990911} 0.829587197672304
{'depth': 8, 'l2_l

KeyboardInterrupt: ignored

In [6]:
#best = {'depth': 6, 'l2_leaf_reg': 1.0, 'learning_rate': 0.07395682681736576} 

model = CatBoostClassifier(
    l2_leaf_reg=4,
    learning_rate=0.22,
    #depth=best['depth'],
    depth=4,
    iterations=1000,
    eval_metric='Accuracy',
    #od_type='Iter',
    #od_wait=40,
    random_seed=42,
    logging_level='Silent',
    allow_writing_files=False
)

cv_data = cv(
    train_pool,
    model.get_params(),
    fold_count=5
)

print('Best validation accuracy score: {:.2f}±{:.2f} on step {}'.format(
    np.max(cv_data['test-Accuracy-mean']), 
    cv_data['test-Accuracy-std'][cv_data['test-Accuracy-mean'].idxmax(axis=0)],
    cv_data['test-Accuracy-mean'].idxmax(axis=0)
))
print('Precise validation accuracy score: {}'.format(np.max(cv_data['test-Accuracy-mean'])))

model.fit(train_pool);
model.score(X_test, y_test)

Best validation accuracy score: 0.83±0.04 on step 93
Precise validation accuracy score: 0.8281560283687943


0.8192090395480226

In [7]:
from sklearn.metrics import confusion_matrix
y_pred_test = model.predict(X_test)
confusion_matrix = pd.DataFrame(confusion_matrix(y_test, y_pred_test))

confusion_matrix.index = ['Actual Died','Actual Survived']
confusion_matrix.columns = ['Predicted Died','Predicted Survived']
print(confusion_matrix)


                 Predicted Died  Predicted Survived
Actual Died                  89                  14
Actual Survived              18                  56


In [8]:
feature_importances = model.get_feature_importance(train_pool)
feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

Fare: 30.35897771629609
Age: 22.17479272549449
GenderClass_male: 10.615354224738613
Pclass: 8.759870743640256
Title: 7.478242474207159
GenderClass_female: 6.543476314505574
FamilySize: 5.384969667615555
Embarked_S: 2.390103608923247
Cabin_B: 1.3932607058964905
Cabin_E: 1.290700066968202
Cabin_X: 0.9776377876917602
Embarked_Q: 0.7642002712847398
Cabin_C: 0.6303382487561273
Cabin_D: 0.438633170583408
Cabin_A: 0.42296892394854013
Cabin_F: 0.17984879175458876
Cabin_G: 0.1779792210284332
Cabin_T: 0.018645336666731144


In [11]:
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier
import lightgbm as lgbm

# Take a hold out set randomly
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an LGBM dataset for training
categorical_features = ['Pclass']
train_data = lgbm.Dataset(data=X_train, label=y_train, categorical_feature=categorical_features, free_raw_data=False)

# Create an LGBM dataset from the test
test_data = lgbm.Dataset(data=X_test, label=y_test, categorical_feature=categorical_features, free_raw_data=False)

# Finally, create a dataset for the FULL training data to give us maximum amount of data to train on after 
# performance has been calibrate
final_train_set = lgbm.Dataset(data=X, label=y, 
                               categorical_feature=categorical_features, free_raw_data=False)

# lgbm_params = {
#     'boosting': 'dart',          # dart (drop out trees) often performs better
#     'application': 'binary',     # Binary classification
#     'learning_rate': 0.05,       # Learning rate, controls size of a gradient descent step
#     'min_data_in_leaf': 20,      # Data set is quite small so reduce this a bit
#     'feature_fraction': 0.7,     # Proportion of features in each boost, controls overfitting
#     'num_leaves': 41,            # Controls size of tree since LGBM uses leaf wise splits
#     'metric': 'binary_logloss',  # Area under ROC curve as the evaulation metric
#     'drop_rate': 0.15
#               }

# modelLGBM = modelLGBM(train_set=train_data,
#                  params=lgbm_params,
#                  valid_sets=[train_data, test_data], 
#                  valid_names=['Train', 'Test'],
#                  evals_result=evaluation_results,
#                  num_boost_round=500,
#                  early_stopping_rounds=100,
#                  verbose_eval=20)

param_grid = {'n_estimators':[50],'max_depth':[4], 'num_boost_round':[500], 'boosting_type':['rf']} 
modelLGBM = LGBMClassifier(params=param_grid)
# param_grid = {'n_estimators':[50],'max_depth':[4,], 'num_boost_round':[500], 'boosting_type':['rf']} 
# grs = GridSearchCV(modelLGBM, param_grid=param_grid, cv = 10, n_jobs=-1, return_train_score = False)
# grs.fit(X_train, y_train)

modelLGBM.fit(X_train, y_train)
# print("Best parameters " + str(grs.best_params_))
# gpd = pd.DataFrame(grs.cv_results_)
# print("Estimated accuracy of this model for unseen data:{0:1.4f}".format(gpd['mean_test_score'][grs.best_index_]))

TypeError: ignored